In [1]:
# !pip3 install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter lab clean
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [185]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML

import re
from dataclasses import dataclass
from typing import List

In [186]:
@dataclass
class Annotation:
    text: str
    start: int
    end: int
        
@dataclass
class Sentence:
    text: str
    annotations: List[Annotation]

In [187]:
def f(x):
    return x

output = widgets.Output()
with output:
    interact(f, x=10)

In [198]:
sentences = ['react is good', 'vue is good too', 'I love React and Vue']
tag = 'SKILL'
keywords = ['react', 'vue']
result: List[Sentence] = []
current_sentence = None

# For adding new keyword.
keyword = ''

output = [None] * len(sentences)
idx = -1

def to_next():
    global idx
    if idx >= len(sentences) - 1:
        return
    idx = idx + 1

def to_prev():
    global idx
    if idx <= 0:
        return
    idx = idx - 1
    
def reset():
    global idx
    global result
    result = []
    idx = -1

def is_end():
    global idx
    return idx == len(sentences) - 1

def accept():
    global current_sentence
    if current_sentence:
        result.append(current_sentence)
    current_sentence = None
    to_next()

def reject():
    pass

def preprocess(sentence):
    global keywords
    global current_sentence
    pattern = '|'.join(keywords)
    annotations: List[Annotation] = []
        
    for m in re.finditer(pattern, sentence, flags=re.IGNORECASE|re.MULTILINE|re.UNICODE):
        s, e = m.span()
        annotations.append(Annotation(sentence[s:e], s, e))
    current_sentence = Sentence(sentence, annotations)
    return current_sentence

In [199]:
button_prev = widgets.Button(description="Prev")
button_next = widgets.Button(description="Next")
button_reset = widgets.Button(description='Reset')
button_add = widgets.Button(description='Add')

def handle_prev(e):
    global idx
    to_prev()
    render()

def handle_next(e):
    accept()
    render()

def handle_reset(e):
    reset()
    render()
    
def handle_add(e):
    global keywords
    global keyword
    if keyword == "": 
        return
    if keyword in keywords:
        keywords.remove(keyword)
    else:
        keywords.append(keyword)
    keyword = ''
    render()

button_prev.on_click(handle_prev)
button_next.on_click(handle_next)
button_reset.on_click(handle_reset)
button_add.on_click(handle_add)

html = widgets.HTML(value='', placeholder='Enter html', description='')

display(button_prev, 
        button_next, 
        button_reset, 
        button_add, html)

def render():
    global idx
    global keywords
    if idx < 0:
        html.value = "Press next to start"
        return
    result = preprocess(sentences[idx])
    sentence = sentences[idx]
    for annotation in result.annotations:
        sentence = sentence.replace(annotation.text, f'<b style="background: #FFFBCC">{annotation.text}</b>', 1)
    
    html.value = f"Total: {len(sentences)}, Current: {idx+1}"
    html.value += '<br/>'
    html.value += "Keywords: " + "|".join(keywords)
    html.value += '<br/>'
    html.value += f'{idx+1}. {sentence}'
    html.value += '<br/>'
    html.value += str(result)
        
def update_keywords(new_keyword):
    global keyword
    keyword = new_keyword
    
interact(update_keywords, new_keyword=keyword)

Button(description='Prev', style=ButtonStyle())

Button(description='Next', style=ButtonStyle())

Button(description='Reset', style=ButtonStyle())

Button(description='Add', style=ButtonStyle())

HTML(value='', placeholder='Enter html')

interactive(children=(Text(value='', description='new_keyword'), Output()), _dom_classes=('widget-interact',))

<function __main__.update_keywords(new_keyword)>

In [204]:
for sent in result:
    print(sent)
    print()

Sentence(text='react is good', annotations=[Annotation(text='react', start=0, end=5)])

Sentence(text='vue is good too', annotations=[Annotation(text='vue', start=0, end=3)])

Sentence(text='I love React and Vue', annotations=[Annotation(text='React', start=7, end=12), Annotation(text='Vue', start=17, end=20)])

